# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]

# log-scaling
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# min-max scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns= ["price"])

## Train test split

**Perform a train-test-split with a test set of 0.20.**

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


**Fit the model and apply the model to the make test set predictions**

In [5]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [6]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

**Calculate the residuals and the mean squared error**

In [7]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

from sklearn.metrics import mean_squared_error
sk_mse_train = mean_squared_error(y_train, y_hat_train)
sk_mse_test = mean_squared_error(y_test, y_hat_test)
print('Mean Squared Error Training Set: ', round(sk_mse_train, 5))
print('Mean Squared Error Test Set:     ', round(sk_mse_test, 5))

Mean Squared Error Training Set:  21.89804
Mean Squared Error Test Set:      21.42649


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [12]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    
    records = len(data)
    # calculates floor division...
    fold_size = records // k
    # ...calculates rest
    leftovers = records % k
    
    folds = []
    start = 0
    
    # Fold Size will be 1 larger to account for leftovers
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:        
            fold = data.iloc[start : start + fold_size+1] 
            folds.append(fold)
            start +=  fold_size + 1
        else:
            fold = data.iloc[start : start + fold_size] 
            folds.append(fold)
            start += fold_size
            
    return folds

### Apply it to the Boston Housing Data

In [13]:
# Make sure to concatenate the data again
boston_df = pd.concat([X.reset_index(drop=True), y], axis=1)

In [14]:
boston_folds = kfolds(boston_df, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [18]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(boston_folds) if i!=n])
    test = boston_folds[n]
    
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    
    # Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))

print(train_errs)
print(test_errs)

[price    24.195577
dtype: float64, price    23.032087
dtype: float64, price    19.745073
dtype: float64, price    15.317101
dtype: float64, price    22.329973
dtype: float64]
[price    13.405145
dtype: float64, price    17.444017
dtype: float64, price    37.032711
dtype: float64, price    58.279544
dtype: float64, price    26.097989
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [20]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [22]:
np.mean(cv_5_results)

-30.451881143540312

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!